In [44]:
import os
import json
import pandas as pd
import requests
import time
import datetime
from datetime import date

In [66]:
# date
after_start_date = date.fromisoformat('2016-04-01')
before_end_date = date.fromisoformat('2016-04-30')
after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()

print('after_start_seconds: ', after_start_seconds)
print('before_end_seconds: ', before_end_seconds)

after_start_seconds:  1459468800.0
before_end_seconds:  1461974400.0


In [46]:
# Category keyword
synonym_ecig = ['electronic-cigarette', 'electronic cigarette', 'electronic cig', ' e-cig', ' ecig', ' e cig', 'e-cigarette', 'ecigarette', ' e cigarette', 'e-juice', ' ejuice', ' e juice', 'e-liquid', ' eliquid', ' e liquid', ' e-smoke', ' esmoke', ' e smoke', ' vape', 'vaper', 'vaping', 'vape-juice',
                'vape-liquid', ' vapor', 'vaporizer', 'boxmod', 'cloud chaser', 'cloudchaser', 'smoke assist', 'ehookah', 'e-hookah', ' e hookah', 'stillblowingsmoke', 'still blowing smoke', 'cherry tip cigarillo', '#e-cig', '#ecig', '#e cigar', '#ejuice', '#e juice', '#eliquid', '#e liquid', '#vapor', '#ehookah', '#e hookah', 'smoke pod']
# remove ' fin '
brand_ecig = ['juul', 'vaporfi', 'vype pebble', 'v2 cig', 'v2cigs', 'v2 cigs', 'halocigs', ' njoy', 'markten', 'vuse', ' tryst', 'atomizer', 'cartomizer', 'south beach smoke', 'eversmoke', 'joye510', 'joye 510', 'joyetech', 'logicecig', 'smartsmoker', ' mistic', 'smokestiks', '21st century smoke', 'logic black label',
              ' finiti', 'nicotek', 'cigirex', 'ciga&blu', 'cig&blu', 'logic&cig', 'e-swisher', 'e swisher', 'eswisher', 'ezsmoker', 'ez&cig', 'green smoke', 'cigalectric', 'xhale o2', 'xhaleo2', 'cig2o', 'green smart living', 'greensmartliving', 'swisher blk', 'grimmgreen', '#njoy', '#fin ', '#finiti']
policy_ecig = ['sb140', 'sb 140', 'sb24', 'sb 24']
cessation_ecig = ['notblowingsmoke', 'not blowing smoke',
                  'tobaccofreekids', 'notareplacement']

def remove_space_and_conjunction(ecig_keywords):
    temp_keywords_list = []
    for keyword in ecig_keywords:
        keyword = keyword.strip()
        if (' ' in keyword and '#' not in keyword) or ('&' in keyword) or ('-' in keyword and '#' not in keyword):
            continue
        temp_keywords_list.append(keyword)
    return temp_keywords_list

# ecig keywords in total
ecig_keywords = []
ecig_keywords.extend(synonym_ecig)
ecig_keywords.extend(brand_ecig)
ecig_keywords.extend(policy_ecig)
ecig_keywords.extend(cessation_ecig)
ecig_keywords = remove_space_and_conjunction(ecig_keywords)

# San Francisco keywords
SF_keywords = ['San Fancisco', 'SF', 'SFO']

# San Francisco ecig keywords
SF_ecig_keywords = [i+' '+j for i in SF_keywords for j in ecig_keywords]
print('length of SF_ecig_keywords: ', len(SF_ecig_keywords))

length of SF_ecig_keywords:  177


In [57]:
def getPushshiftData_noSubreddit(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
#     print(r)
    data = json.loads(r.text)
    return data['data']

In [48]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
#     author_fullName = subm['author_fullname']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
        
    try:
        self_text = subm['selftext']
    except KeyError:
        self_text = "None"
        
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id, title, self_text, url, author, score,subreddit, created,numComms,permalink,flair))
    subStats[sub_id] = subData

### Search keyword without specifying subreddit

In [74]:
subStats = {}

for query in SF_ecig_keywords:
    print('--------------------------------------')
    print('Query: ', query)
    after = str(after_start_seconds).split('.')[0]
    before = str(before_end_seconds).split('.')[0]



    data = getPushshiftData_noSubreddit(query, after, before)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
#             print(submission)
#             print(submission['title'])
#             print(submission['subreddit'])
#             print('-----------')

            collectSubData(submission)

        # Prepare for next iteration
        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = getPushshiftData_noSubreddit(query, after, before)

    print(len(data))
    print('length of subStats.keys(): ', len(subStats.keys()))
    time.sleep(.300)
    
    
    
    
    
#     print('subreddit_dict: ', subreddit_dict)
#     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))
#     break

# subStats = {}
# # subreddit_dict = {}
# temp = []

# for query in SF_ecig_keywords:
#     print('--------------------------------------')
#     print('Query: ', query)
#     after = str(after_start_seconds).split('.')[0]
#     before = str(before_end_seconds).split('.')[0]

#     data = getPushshiftData_noSubreddit(query, after, before)
    
#     count = 0
#     # Will run until all posts have been gathered from the 'after' date up until before date
#     while len(data) > 0:
#         for submission in data:
#             collectSubData(submission)
# #             print(submission)
# #             print(submission['title'])
# #             print(submission['subreddit'])
# #             print('-----------')

#         # Prepare for next iteration
#         print(len(data))
#         print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
#         after = data[-1]['created_utc']
#         temp.append(after)
#         data = getPushshiftData_noSubreddit(query, after, before)
        
#         count += 1
#         if count%10 == 0:
#             break

#     print(len(data))
# #     print('length of subStats.keys(): ', len(subStats.keys()))
# #     print('subreddit_dict: ', subreddit_dict)
# #     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))
#     time.sleep(.300)
#     # break


--------------------------------------
Query:  San Fancisco ecig
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco ecigarette
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco ejuice
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco eliquid
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco esmoke
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco vape
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco vaper
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco vaping
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco vapor
0
length of subStats.keys():  0
--------------------------------------
Query:  San Fancisco vaporizer
0
length of subStats.keys():  0
--------

25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-08 13:45:42
25
2015-10-

KeyboardInterrupt: 

In [77]:
subStats

{'d5v6to': [('d5v6to',
   'Madlib/Freddie Gibbs 2019 San Fancisco Show Poster by Jason Jagel',
   '',
   'https://www.rappcats.com/shop/madlib-freddie-gibbs-2019-san-fancisco-show-poster-by-jason-jagel/',
   'TheUnseenEra',
   7,
   'Madlib',
   datetime.datetime(2019, 9, 18, 9, 22, 19),
   0,
   '/r/Madlib/comments/d5v6to/madlibfreddie_gibbs_2019_san_fancisco_show_poster/',
   'NaN')],
 'cme6z3': [('cme6z3',
   'Any Galaxy/astronomy/outter space theme tattoo artists in San Fancisco Bay Area?',
   '',
   'https://www.reddit.com/r/tattoos/comments/cme6z3/any_galaxyastronomyoutter_space_theme_tattoo/',
   'sidmendoza',
   2,
   'tattoos',
   datetime.datetime(2019, 8, 5, 17, 41, 33),
   2,
   '/r/tattoos/comments/cme6z3/any_galaxyastronomyoutter_space_theme_tattoo/',
   'NaN')],
 'cmdxpk': [('cmdxpk',
   'Any Galaxy/astronomy/outter space theme tattoo artists in San Fancisco Bay Area?',
   '[removed]',
   'https://www.reddit.com/r/bayarea/comments/cmdxpk/any_galaxyastronomyoutter_space_t

In [76]:
len(subStats)
# subStats['d5v6to']

25

In [75]:
temp

[1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,
 1444311942,